In [ ]:
import os
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np

from functions import getAnalysisNumbers
from runDataClass import runData

In [ ]:
def plotSingleAvalanche(runData, avalancheID, plotName):
    """
    Plots the drift lines of all electrons from a specified simulated electron avalanche.

    The pad geometry and the hole in the grid are included.

    Args:
        runData (class): 
        avalancheID (int): The ID number of the desired avalanche to plot
        plotName (str): A string used to title the plot.

    Returns:
        None
    """

    allAvalanches = runData.getDataFrame('electronTrackData')
    
    # Isolate the desired avalanche
    singleAvalanche = allAvalanches[allAvalanches['Avalanche ID'] == avalancheID]
    gain = singleAvalanche['Electron ID'].max()+1
    
    # Group drift lines together based on the electron ID
    groupedAvalanche = singleAvalanche.groupby('Electron ID')

    if groupedData is None:
        print(f"An error occured plotting avalanche '{avalancheID}'.")
        return None
        
    # Create the figure and add subplots
    fig2D = plt.figure(figsize=(14, 7))
    fig2D.suptitle(f'Avalanche: {plotName} (ID #{avalancheID}) Gain = {gain}')
    
    ax11 = fig.add_subplot(221)
    ax12 = fig.add_subplot(223)
    ax13 = fig.add_subplot(122)

    # Iterate through the drift lines of each electron
    for electronID, driftLine in groupedAvalanche:
        ax11.plot(driftLine['Drift x'], 
                  driftLine['Drift z'], 
                  linewidth=.5)
        ax12.plot(driftLine['Drift y'], 
                  driftLine['Drift z'], 
                  linewidth=.5)
        ax13.plot(driftLine['Drift x'], 
                  driftLine['Drift y'], 
                  linewidth=.5)

    # Plot the initial electron location
    ax11.plot(singleAvalanche['Drift x'].iloc[0], 
              singleAvalanche['Drift z'].iloc[0], 
              label='Initial', marker='x', c='r')
    ax12.plot(singleAvalanche['Drift y'].iloc[0], 
              singleAvalanche['Drift z'].iloc[0], 
              label='Initial', marker='x', c='r')
    ax13.plot(singleAvalanche['Drift x'].iloc[0], 
              singleAvalanche['Drift y'].iloc[0], 
              label='Initial', marker='x', c='r')
    
    #Add geometry elements
    runData._plotAddCellGeometry(ax11, 'xz')
    runData._plotAddCellGeometry(ax12, 'yz')
    runData._plotAddCellGeometry(ax13, 'xy')

    return fig2D

In [ ]:
def plotLocations(particleData, metaData, plotName) -> None:
    """
    Generates 2D histograms to visualize the inital and final locations
    of simulated particles from many avalanches.

    The pad geometry and the hole in the grid are included.

    Args:
        particleData (DataFrame): A Pandas DataFrame containing the initial and final
                                  coordinates for a type of particle.
        metaData (DataFrame): A DataFrame containing metadata about the detector geometry.
        plotName (str): A string used to title the plots.

    Returns:
        None
    """
    # Extract relevant geometric parameters from metadata.
    pitch = metaData['Pitch'].iloc[0]
    halfGridThickness = metaData['Grid Thickness'].iloc[0]/2.
    padHeight = -halfGridThickness - metaData['Grid Standoff'].iloc[0]
    cathodeHeight = halfGridThickness + metaData['Cathode Height'].iloc[0]

    numBins = 51

    # Create the figure and add subplots
    fig = plt.figure(figsize=(12, 12))
    fig.suptitle(f'Particle Locations: {plotName}')
    
    ax1 = fig.add_subplot(221)
    ax2 = fig.add_subplot(222)
    ax3 = fig.add_subplot(223)
    ax4 = fig.add_subplot(224)

    rangeXY = [[-pitch, pitch], [-pitch, pitch]]
    rangeXZ = [[-pitch, pitch], [padHeight, cathodeHeight]]

    # Plot data
    ax1.hist2d(particleData['Initial x'], 
               particleData['Initial y'], 
               bins=numBins, range=rangeXY, cmin=1)
    ax2.hist2d(particleData['Final x'], 
               particleData['Final y'], 
               bins=numBins, range=rangeXY, cmin=1)
    ax3.hist2d(particleData['Initial x'], 
               particleData['Initial z'], 
               bins=numBins, range=rangeXZ, cmin=1)
    ax4.hist2d(particleData['Final x'], 
               particleData['Final z'], 
               bins=numBins, range=rangeXZ, cmin=1)

    #Add geometry Pieces
    plotAddCellGeometry(ax1, 'xy', metaData)
    plotAddCellGeometry(ax2, 'xy', metaData)
    plotAddCellGeometry(ax3, 'xz', metaData)
    plotAddCellGeometry(ax4, 'xz', metaData)

    ax1.set_title('Initial')
    ax2.set_title('Final')
    ax3.set_title('Initial')
    ax4.set_title('Final')


    return

In [ ]:
def plotDiffusion(particleData, metaData, plotName) -> None:
    """
    """
    halfPitch = metaData['Pitch'].iloc[0]/2.
    halfGridThickness = metaData['Grid Thickness'].iloc[0]/2.
    padHeight = -halfGridThickness - metaData['Grid Standoff'].iloc[0]
    cathodeHeight = halfGridThickness + metaData['Cathode Height'].iloc[0]
    
    driftX = particleData['Final x'] - particleData['Initial x']
    driftY = particleData['Final y'] - particleData['Initial y']
    driftZ = particleData['Final z'] - particleData['Initial z']

    driftR = np.sqrt(driftX**2 + driftY**2)

    fig = plt.figure(figsize=(12, 4))
    fig.suptitle(f'{plotName}')
    
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)

    ax1.hist(abs(driftZ))
    ax2.hist(driftR)

    ax1.set_title('Drift in z')
    ax2.set_title('Drift in r')

    ax1.set_xlabel('Distance in z (cm)')
    ax2.set_xlabel('Distance in xy plane (cm)')

    ax1.grid()
    ax2.grid()
    
    return